Python Script to generate Spare Parts Demand for 14th Month by 13 Last Month  
Original Author     : Michael Brandon  
Original Modified   : 2024-12-24  
Modified By         : Unknown Player  
Modified Date       : 2024-12-24  

In [15]:
# PACKAGES
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import PolynomialFeatures
from statsmodels.tsa.api import SimpleExpSmoothing, ExponentialSmoothing
from tabulate import tabulate
import os
import json
import sys
import requests
import math

# Set Display Width Longer
pd.options.display.max_colwidth = 100 # 100 for long width

In [ ]:
# Retrive JSON Data From API.
url = "http://172.16.1.59:18080/v1/web/get-spare-parts-all-history-data"

# Fetch JSON data from the API
response = requests.get(url)
response.raise_for_status()  # Raise an error if the request fails
df = response.json()  # Parse JSON data

# Convert JSON to Pandas DataFrame
data = pd.DataFrame(df['data'][:5])
display(data)

In [27]:
# Add Metric to the data

# get mean and standart deviation of first 12 data
data['mean_12'] = data['D'].apply(lambda x: np.mean(x[:12]))
data['std_12'] = data['D'].apply(lambda x: np.std(x[:12], ddof=1))

# get upper bound from mean and std
data['ub'] = data['mean_12'] + 1.5 * data['std_12']

# limit the original data to upper bound
data['clipped_d'] = data.apply(lambda row: np.clip(row['D'][:12], 0, row['ub']).tolist(), axis=1)
display(data)

,branch,agency,partno,D,FD,mean_12,std_12,ub,clipped_d
0,99,99,ZZ01,"[11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23]",0.0,16.500000,3.605551,21.908327,"[11.0, 12.0, 13.0, 14.0, 15.0, 16.0, 17.0, 18.0, 19.0, 20.0, 21.0, 21.908326913195985]"
1,99,99,ZZ02,"[22, 23, 34, 45, 56, 67, 78, 89, 90, 11, 22, 33, 23]",0.0,47.500000,27.868196,89.302294,"[22.0, 23.0, 34.0, 45.0, 56.0, 67.0, 78.0, 89.0, 89.30229441288861, 11.0, 22.0, 33.0]"
2,99,99,ZZ03,"[33, 67, 77, 88, 99, 10, 21, 32, 43, 54, 65, 76, 87]",0.0,55.416667,27.848481,97.189388,"[33.0, 67.0, 77.0, 88.0, 97.18938775678393, 10.0, 21.0, 32.0, 43.0, 54.0, 65.0, 76.0]"
3,99,99,ZZ04,"[44, 10, 20, 30, 40, 50, 60, 70, 80, 90, 23, 34, 12]",0.0,45.916667,24.901473,83.268875,"[44.0, 10.0, 20.0, 30.0, 40.0, 50.0, 60.0, 70.0, 80.0, 83.26887543669793, 23.0, 34.0]"
4,99,99,ZZ05,"[55, 25, 22, 23, 35, 66, 12, 23, 34, 45, 56, 67, 78]",0.0,38.583333,18.729939,66.678241,"[55.0, 25.0, 22.0, 23.0, 35.0, 66.0, 12.0, 23.0, 34.0, 45.0, 56.0, 66.67824147762816]"


In [44]:
# Calculate Moving Average
for row in data['clipped_d']:
    for i in row:
        print(i)
    print(np.mean(row))

# Display the result
display(data)

11.0
12.0
13.0
14.0
15.0
16.0
17.0
18.0
19.0
20.0
21.0
21.908326913195985
16.492360576099667
22.0
23.0
34.0
45.0
56.0
67.0
78.0
89.0
89.30229441288861
11.0
22.0
33.0
47.441857867740715
33.0
67.0
77.0
88.0
97.18938775678393
10.0
21.0
32.0
43.0
54.0
65.0
76.0
55.26578231306533
44.0
10.0
20.0
30.0
40.0
50.0
60.0
70.0
80.0
83.26887543669793
23.0
34.0
45.35573961972483
55.0
25.0
22.0
23.0
35.0
66.0
12.0
23.0
34.0
45.0
56.0
66.67824147762816
38.55652012313568


,branch,agency,partno,D,FD,mean_12,std_12,ub,clipped_d,ma
0,99,99,ZZ01,"[11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23]",0.0,16.500000,3.605551,21.908327,"[11.0, 12.0, 13.0, 14.0, 15.0, 16.0, 17.0, 18.0, 19.0, 20.0, 21.0, 21.908326913195985]","[nan, 11.5, 12.5, 13.5, 14.5, 15.5, 16.5, 17.5, 18.5, 19.5, 20.5, 21.454163456597993]"
1,99,99,ZZ02,"[22, 23, 34, 45, 56, 67, 78, 89, 90, 11, 22, 33, 23]",0.0,47.500000,27.868196,89.302294,"[22.0, 23.0, 34.0, 45.0, 56.0, 67.0, 78.0, 89.0, 89.30229441288861, 11.0, 22.0, 33.0]","[nan, 22.5, 28.5, 39.5, 50.5, 61.5, 72.5, 83.5, 89.1511472064443, 50.15114720644431, 16.5, 27.5]"
2,99,99,ZZ03,"[33, 67, 77, 88, 99, 10, 21, 32, 43, 54, 65, 76, 87]",0.0,55.416667,27.848481,97.189388,"[33.0, 67.0, 77.0, 88.0, 97.18938775678393, 10.0, 21.0, 32.0, 43.0, 54.0, 65.0, 76.0]","[nan, 50.0, 72.0, 82.5, 92.59469387839196, 53.594693878391965, 15.5, 26.5, 37.5, 48.5, 59.5, 70.5]"
3,99,99,ZZ04,"[44, 10, 20, 30, 40, 50, 60, 70, 80, 90, 23, 34, 12]",0.0,45.916667,24.901473,83.268875,"[44.0, 10.0, 20.0, 30.0, 40.0, 50.0, 60.0, 70.0, 80.0, 83.26887543669793, 23.0, 34.0]","[nan, 27.0, 15.0, 25.0, 35.0, 45.0, 55.0, 65.0, 75.0, 81.63443771834896, 53.13443771834896, 28.5]"
4,99,99,ZZ05,"[55, 25, 22, 23, 35, 66, 12, 23, 34, 45, 56, 67, 78]",0.0,38.583333,18.729939,66.678241,"[55.0, 25.0, 22.0, 23.0, 35.0, 66.0, 12.0, 23.0, 34.0, 45.0, 56.0, 66.67824147762816]","[nan, 40.0, 23.5, 22.5, 29.0, 50.5, 39.0, 17.5, 28.5, 39.5, 50.5, 61.33912073881408]"
